In [1]:
from IPython.display import Image
from pathlib import Path
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
import math
import xgboost as xgb

In [2]:
train_labels = pd.read_csv("KD/train_labels_KD.csv", index_col="seq_id")
train_labels_columns=train_labels.columns
train_labels['season'] = train_labels.index.map(lambda x: x.split("#")[0])


In [3]:
feature_classes=list(train_labels_columns)
feature_classes.remove('reptiles')
feature_classes.remove('caracal')
feature_classes.remove('wildcat')
feature_classes.remove('honeybadger')
feature_classes.remove('vulture')
feature_classes.remove('duiker')
feature_classes.remove('civet')
feature_classes.remove('genet')
feature_classes.remove('rhinoceros')
feature_classes.remove('zorilla')
feature_classes.remove('cattle')
feature_classes.remove('steenbok')
feature_classes.remove('bat')


In [4]:
everything_S10 = pd.read_csv("KD/everything_fullhd_SER_S10_all.csv", index_col="seq_id")
everything_S9 = pd.read_csv("KD/everything_fullhd_SER_S9_all.csv", index_col="seq_id_buf")
everything_S9.index.name='seq_id'

# everything_s10 = pd.read_csv("KD/everything_SER_S10.csv", index_col="seq_id_buf")
# everything_s10.index.name='seq_id'
len(everything_S10), len(everything_S9)

(252011, 367394)

In [5]:
len(set(everything_S10.index)), len(set(everything_S9.index)), 

(252011, 367394)

In [11]:
everything = pd.concat([everything_S9,  everything_S10])


In [12]:
join = everything.join(train_labels, how='left')


In [13]:
len(join)

619405

In [14]:
len(join[join['season']=='SER_S9'])

367394

In [15]:
len(join[join['season']=='SER_S10'])

252011

In [16]:
models = [
            'pred_model_480_360_as512_img',
            'pred_model_480_360_as480_img',
            'pred_model_480_360_v2_as512_img', 
            'pred_model_480_360_v2_as480_img', 
            'pred_model_512_384_v2_as512_img'
         ]
models2 = [
           'pred_mean_back_2models_as512_',
           'pred_mean_back_2models_as480_',
          ]

models3 = [
           'pred_back_as480_',
           'pred_back_as512_',
          ]


In [17]:
interesting_columns = ['count_imgs', 'season']
models1 = []
for m in models:
    models1.append(m + '1_')
    models1.append(m + '2_')
    models1.append(m + '3_')
for m in models1 + models2 + models3 + [""]:
    for c in feature_classes:
        interesting_columns.append(m + c)
        

In [18]:
join=join[interesting_columns]

In [19]:
# for model_id in ['pred_model_480_360_v2_as480_img1_', 'pred_model_480_360_v2_as480_img1_','pred_model_480_360_v2_as480_img1_',
#          'pred_model_480_360_v2_as512_img1_', 'pred_model_480_360_v2_as512_img2_','pred_model_480_360_v2_as512_img3_',
#          'pred_model_480_360_as480_img1_', 'pred_model_480_360_as480_img2_','pred_model_480_360_as480_img3_',
#          'pred_model_480_360_as512_img1_', 'pred_model_480_360_as512_img2_','pred_model_480_360_as512_img3_',
#           'pred_mean_back_2models_as512_','pred_back_as512_']:
#     join[model_id + 'nonempty'] = join[[model_id + c for c in list(train_labels.columns) if c != 'empty']].sum(axis=1)
#     join[model_id + 'maxnonempty'] = join[[model_id + c for c in list(train_labels.columns) if c != 'empty']].max(axis=1)

In [20]:
for c in feature_classes:
    print(c)
    join['median_img1_' + c] = join[[m + '1_' + c for m in models]].median(axis=1)
    join['median_img2_' + c]=join['median_img1_' + c]
    join['median_img3_' + c]=join['median_img1_' + c]
    join.loc[join.count_imgs > 1, 'median_img2_' + c] = join.loc[join.count_imgs > 1, [m + '2_' + c for m in models]].median(axis=1)
    join.loc[join.count_imgs > 2, 'median_img3_' + c] = join.loc[join.count_imgs > 2, [m + '3_' + c for m in models]].median(axis=1)
    join['mean_imgs_' + c] = (join['median_img1_' + c] + join['median_img2_' + c]*0.8 + join['median_img3_' + c]*0.5) /2.3
    
    join['mean_combines_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_combines_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models2]].median(axis=1)
    
    join['mean_backs_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_backs_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models3]].median(axis=1)
    

aardvark
aardwolf
baboon
batearedfox
buffalo
bushbuck
cheetah
dikdik
eland
elephant
empty
gazellegrants
gazellethomsons
giraffe
guineafowl
hare
hartebeest
hippopotamus
hyenaspotted
hyenastriped
impala
insectspider
jackal
koribustard
leopard
lionfemale
lionmale
mongoose
monkeyvervet
ostrich
otherbird
porcupine
reedbuck
rodents
secretarybird
serval
topi
warthog
waterbuck
wildebeest
zebra


In [21]:
DATA_PATH = Path("../../datasets/wildlife") 
train_metadata = pd.read_csv(DATA_PATH / "train_metadata.csv", parse_dates= ['datetime'], index_col="seq_id")
train_metadata = train_metadata.groupby('seq_id').first()
train_metadata['month']=train_metadata.datetime.dt.month
train_metadata['hour']=train_metadata.datetime.dt.hour
join = join.join(train_metadata.hour, how='left')
join = join.join(train_metadata.month, how='left')


In [22]:

join.loc[join.count_imgs<2, ['pred_back_as480_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_back_as512_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_mean_back_2models_as480_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_mean_back_2models_as512_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_as480_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_as480_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_as512_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes]]=-0.01

           



           

In [23]:

test_index = set(join[join.season=='SER_S9'].index[::2])
test_dataset     = join[join.index.isin(test_index)]
val_dataset = join[(join.season=='SER_S9') & (~join.index.isin(test_index))]
train_dataset = join[join.season=='SER_S10']
animal_columns =list(train_labels.columns)
animal_columns.remove('empty')

# test_index = set(join[join.season=='SER_S10'].index[::5])
# test_dataset = join[join.index.isin(test_index)]
# val_index = set(join[(join.season=='SER_S10') & (~join.index.isin(test_index))].index[::4])
# val_dataset = join[join.index.isin(val_index)]
# train_dataset = join[(~join.index.isin(test_index)) & (~join.index.isin(val_index))]



# (699208, 76659, 76660)
len(train_dataset), len(val_dataset), len(test_dataset)

(252011, 183697, 183697)

In [46]:
training_distibution

empty              0.785593
wildebeest         0.095885
zebra              0.045871
gazellethomsons    0.024674
buffalo            0.007646
elephant           0.006889
hartebeest         0.003670
impala             0.005412
gazellegrants      0.003163
giraffe            0.004282
warthog            0.004234
guineafowl         0.002972
otherbird          0.003345
hyenaspotted       0.002488
lionfemale         0.001952
eland              0.001286
hippopotamus       0.001095
reedbuck           0.000659
topi               0.001270
baboon             0.000881
dikdik             0.001678
secretarybird      0.000306
jackal             0.000714
dtype: float64

In [47]:
test_distibution

empty              0.759851
wildebeest         0.076248
zebra              0.053718
gazellethomsons    0.037591
buffalo            0.009228
elephant           0.008200
hartebeest         0.008055
impala             0.006635
gazellegrants      0.006394
giraffe            0.006195
warthog            0.005640
guineafowl         0.004460
otherbird          0.004275
hyenaspotted       0.004108
lionfemale         0.002366
eland              0.002281
hippopotamus       0.002017
reedbuck           0.001916
topi               0.001663
baboon             0.001154
dikdik             0.001125
secretarybird      0.000695
jackal             0.000531
dtype: float64

In [24]:
import numpy as np
lgbt_models = ['empty', 'wildebeest', 'zebra', 'gazellethomsons', 'buffalo',
'elephant', 'hartebeest', 'impala', 'gazellegrants', 'giraffe',
'warthog', 'guineafowl', 'otherbird', 'hyenaspotted', 'lionfemale', 
'eland', 'hippopotamus', 'reedbuck', 'topi', 'baboon', 'dikdik', 
'secretarybird', 'jackal']

# training_distibution = join.loc[train_labels.season.isin(['SER_S9']), lgbt_models].mean(axis=0)
training_distibution = train_dataset[lgbt_models].mean(axis=0)

test_distibution = train_labels[lgbt_models].mean(axis=0)

scale_pos_weight=test_distibution/training_distibution

for key, value in scale_pos_weight.items():
    # do something with value
    scale_pos_weight[key] = np.clip(value,0.6,1.8)
scale_pos_weight

empty              0.967233
wildebeest         0.795202
zebra              1.171072
gazellethomsons    1.523527
buffalo            1.206831
elephant           1.190403
hartebeest         1.800000
impala             1.225881
gazellegrants      1.800000
giraffe            1.446917
warthog            1.332141
guineafowl         1.500770
otherbird          1.277891
hyenaspotted       1.651315
lionfemale         1.212154
eland              1.773820
hippopotamus       1.800000
reedbuck           1.800000
topi               1.309297
baboon             1.309670
dikdik             0.670112
secretarybird      1.800000
jackal             0.743750
dtype: float64

In [34]:
# empty              0.967163
# wildebeest         1.155221
# zebra              0.936143
# gazellethomsons    0.953444
# buffalo            1.661107
# elephant           1.265842
# hartebeest         1.800000
# impala             1.763873
# gazellegrants      1.592588
# giraffe            1.443265
# warthog            1.201960
# guineafowl         1.800000
# otherbird          1.248398
# hyenaspotted       0.902223
# lionfemale         1.354255
# eland              1.800000
# hippopotamus       0.983885
# reedbuck           1.325607
# topi               1.011262
# baboon             0.992658
# dikdik             1.190888
# secretarybird      0.810593
# jackal             0.657136

In [25]:
from sklearn.metrics import log_loss
loss = 0
for c in feature_classes:
    x = test_dataset[['mean_imgs_' + c, c]].values
    if sum(x[:,1]) !=0:
#         print(c)
        l = log_loss(x[:,1], x[:,0])
        loss+=l
        print(c, l)
print(loss/54)

aardvark 0.00018717794993557049
aardwolf 0.00036252127883297465
baboon 0.0012423200714669859
batearedfox 0.0006758061824036838
buffalo 0.0038565876039087297
bushbuck 0.00011946148168219626
cheetah 0.0010419235919881748
dikdik 0.0009244857180794508
eland 0.0010404325222455736
elephant 0.003506064195273324
empty 0.05224119934066942
gazellegrants 0.007001535141392664
gazellethomsons 0.01770252024358524
giraffe 0.0026530418008982944
guineafowl 0.0010869217431119203
hare 0.000386163613811703
hartebeest 0.0031658027577114323
hippopotamus 0.00046265635091280227
hyenaspotted 0.002618244223054782
hyenastriped 0.0003063995077654593
impala 0.0038307071729338196
insectspider 0.002976301380462903
jackal 0.0012740444076699887
koribustard 0.0011475564585479098
leopard 0.00022340565615609794
lionfemale 0.002302318746269777
lionmale 0.0009343714347036501
mongoose 0.0007375214091515641
monkeyvervet 0.00042552325812320437
ostrich 0.0013242010380726476
otherbird 0.011948464322505225
porcupine 0.0001078684

In [28]:
import gc
del train_metadata
del everything_S10
del everything_S9
del train_labels
del join
gc.collect()

52

In [26]:


num_leaves={

}

max_depth={


}
lambda_l1={
    'empty': 0.3,
    'zebra': 0.2,
    'buffalo': 0.3,
}
lambda_l2={
    'empty': 0.3,
    'zebra': 0.2,
    'buffalo': 0.3,
}

learning_rate={
    'zebra':0.008,
    'empty': 0.008,
}

feature_fraction={
    'zebra':0.5,
}
bagging_fraction={
    'zebra':0.8,
    'empty':0.8, #?
}
bagging_freq={
    'zebra':5,
    'empty':5, #?
}
feature_fraction={
    'zebra':0.6,
    'empty':0.3,
}
# elephant
min_child_samples={
    'empty':30,
    'zebra':30,
    'buffalo':30,
    'topi':30,
    'wildebeest':30,
    'gazellethomsons':30
}

steps={
    'empty' : int(907*1.1),
    'wildebeest' : int(865*1.1),
    'zebra' : int(1050*1.1),
    'gazellethomsons' : int(1072*1.1),
    'buffalo' : int(589*1.1),
    'elephant' : int(563*1.1),
    'hartebeest' : int(620*1.1),
    'impala' : int(630*1.1),
    'gazellegrants' : int(616*1.1),
    'giraffe' : int(705*1.1),
    'warthog' : int(752*1.1),
    'guineafowl' : int(416*1.1),
    'otherbird' : int(348*1.1),


}
clazz='wildebeest'

columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']


# columns = ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
#     ['mean_backs_' + c for c in feature_classes] +\
#     ['median_img1_' + c for c in feature_classes] +\
#     ['median_img2_' + c for c in feature_classes] +\
#     ['median_img3_' + c for c in feature_classes] +\
#     ['mean_combines_' + c for c in feature_classes] +\
#     ['hour', 'month']

In [38]:


# X_train = pd.concat([train_dataset[columns], test_dataset[columns]])
# y_train = pd.concat([train_dataset[clazz], test_dataset[clazz]])
X_train = train_dataset[columns]
y_train = train_dataset[clazz]
X_val = val_dataset[columns]
y_val = val_dataset[clazz]

In [39]:
len(X_train)

252011

In [40]:
clazz

'wildebeest'

In [41]:
y_train.mean()


0.09588470344548453

In [48]:
y_val.mean()


0.06606531407698547

In [49]:
y_val.mean()/y_train.mean()


0.6890078573851675

In [43]:
scale_pos_weight['warthog']

1.332140851147961

In [45]:


lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

scale = y_train.mean()
print(clazz, math.log(1/scale))
params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': {'binary_logloss'},
                'num_leaves': num_leaves.get(clazz, 32),
                'max_depth': max_depth.get(clazz, 6),
                'learning_rate': learning_rate.get(clazz, 0.009),
                'feature_fraction':  feature_fraction.get(clazz, 0.6),
                'bagging_fraction': bagging_fraction.get(clazz, 0.8),
                'bagging_freq': bagging_freq.get(clazz, 5),
                'scale_pos_weight':1,
                'lambda_l1':lambda_l1.get(clazz, 0.2),
                'lambda_l2':lambda_l2.get(clazz, 0.2),
                'min_child_samples':min_child_samples.get(clazz, 25),

}


gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=3000,
                    valid_sets=[lgb_train, lgb_val],
                    early_stopping_rounds=20)
gbm.save_model("lgbs_v12/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

X_test = test_dataset[columns]
y_test = test_dataset[clazz]

print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
# Early stopping, best iteration is:
# [976]	training's binary_logloss: 0.0121356	valid_1's binary_logloss: 0.0153744
# wildebeest 0.015022796309252809

wildebeest 2.3446088150759112
[1]	training's binary_logloss: 0.307599	valid_1's binary_logloss: 0.243005
Training until validation scores don't improve for 20 rounds
[2]	training's binary_logloss: 0.299925	valid_1's binary_logloss: 0.237432
[3]	training's binary_logloss: 0.292807	valid_1's binary_logloss: 0.232261
[4]	training's binary_logloss: 0.286169	valid_1's binary_logloss: 0.227426
[5]	training's binary_logloss: 0.279948	valid_1's binary_logloss: 0.222878
[6]	training's binary_logloss: 0.27409	valid_1's binary_logloss: 0.218584
[7]	training's binary_logloss: 0.268558	valid_1's binary_logloss: 0.214518
[8]	training's binary_logloss: 0.263317	valid_1's binary_logloss: 0.21065
[9]	training's binary_logloss: 0.258333	valid_1's binary_logloss: 0.206969
[10]	training's binary_logloss: 0.253587	valid_1's binary_logloss: 0.203452
[11]	training's binary_logloss: 0.249049	valid_1's binary_logloss: 0.200084
[12]	training's binary_logloss: 0.244705	valid_1's binary_logloss: 0.196845
[13]	tra

[108]	training's binary_logloss: 0.0856961	valid_1's binary_logloss: 0.0725978
[109]	training's binary_logloss: 0.0849849	valid_1's binary_logloss: 0.0720153
[110]	training's binary_logloss: 0.0842837	valid_1's binary_logloss: 0.0714433
[111]	training's binary_logloss: 0.083591	valid_1's binary_logloss: 0.0708769
[112]	training's binary_logloss: 0.0829048	valid_1's binary_logloss: 0.070316
[113]	training's binary_logloss: 0.0822286	valid_1's binary_logloss: 0.0697651
[114]	training's binary_logloss: 0.0815605	valid_1's binary_logloss: 0.069218
[115]	training's binary_logloss: 0.0809009	valid_1's binary_logloss: 0.0686815
[116]	training's binary_logloss: 0.0802458	valid_1's binary_logloss: 0.0681474
[117]	training's binary_logloss: 0.079601	valid_1's binary_logloss: 0.0676181
[118]	training's binary_logloss: 0.0789637	valid_1's binary_logloss: 0.0670964
[119]	training's binary_logloss: 0.0783329	valid_1's binary_logloss: 0.0665793
[120]	training's binary_logloss: 0.0777084	valid_1's bin

[212]	training's binary_logloss: 0.0414231	valid_1's binary_logloss: 0.0362441
[213]	training's binary_logloss: 0.0411851	valid_1's binary_logloss: 0.0360493
[214]	training's binary_logloss: 0.0409492	valid_1's binary_logloss: 0.0358567
[215]	training's binary_logloss: 0.0407168	valid_1's binary_logloss: 0.0356647
[216]	training's binary_logloss: 0.0404864	valid_1's binary_logloss: 0.0354758
[217]	training's binary_logloss: 0.0402588	valid_1's binary_logloss: 0.0352888
[218]	training's binary_logloss: 0.0400319	valid_1's binary_logloss: 0.0351034
[219]	training's binary_logloss: 0.0398081	valid_1's binary_logloss: 0.0349181
[220]	training's binary_logloss: 0.0395862	valid_1's binary_logloss: 0.0347363
[221]	training's binary_logloss: 0.0393668	valid_1's binary_logloss: 0.0345579
[222]	training's binary_logloss: 0.0391498	valid_1's binary_logloss: 0.0343782
[223]	training's binary_logloss: 0.0389339	valid_1's binary_logloss: 0.0342001
[224]	training's binary_logloss: 0.0387199	valid_1's

[318]	training's binary_logloss: 0.0252827	valid_1's binary_logloss: 0.0232532
[319]	training's binary_logloss: 0.0251912	valid_1's binary_logloss: 0.0231835
[320]	training's binary_logloss: 0.0251006	valid_1's binary_logloss: 0.023115
[321]	training's binary_logloss: 0.0250115	valid_1's binary_logloss: 0.0230464
[322]	training's binary_logloss: 0.0249223	valid_1's binary_logloss: 0.0229774
[323]	training's binary_logloss: 0.0248343	valid_1's binary_logloss: 0.0229106
[324]	training's binary_logloss: 0.0247461	valid_1's binary_logloss: 0.0228446
[325]	training's binary_logloss: 0.0246597	valid_1's binary_logloss: 0.0227801
[326]	training's binary_logloss: 0.0245733	valid_1's binary_logloss: 0.0227144
[327]	training's binary_logloss: 0.0244877	valid_1's binary_logloss: 0.022647
[328]	training's binary_logloss: 0.0244024	valid_1's binary_logloss: 0.022582
[329]	training's binary_logloss: 0.0243187	valid_1's binary_logloss: 0.022516
[330]	training's binary_logloss: 0.0242356	valid_1's bin

[423]	training's binary_logloss: 0.0188106	valid_1's binary_logloss: 0.0184676
[424]	training's binary_logloss: 0.0187696	valid_1's binary_logloss: 0.0184413
[425]	training's binary_logloss: 0.0187318	valid_1's binary_logloss: 0.0184147
[426]	training's binary_logloss: 0.0186926	valid_1's binary_logloss: 0.018388
[427]	training's binary_logloss: 0.0186538	valid_1's binary_logloss: 0.0183615
[428]	training's binary_logloss: 0.0186175	valid_1's binary_logloss: 0.0183365
[429]	training's binary_logloss: 0.0185805	valid_1's binary_logloss: 0.0183107
[430]	training's binary_logloss: 0.0185445	valid_1's binary_logloss: 0.0182862
[431]	training's binary_logloss: 0.0185079	valid_1's binary_logloss: 0.0182611
[432]	training's binary_logloss: 0.0184716	valid_1's binary_logloss: 0.0182361
[433]	training's binary_logloss: 0.0184355	valid_1's binary_logloss: 0.0182119
[434]	training's binary_logloss: 0.0184	valid_1's binary_logloss: 0.0181881
[435]	training's binary_logloss: 0.0183634	valid_1's bin

[528]	training's binary_logloss: 0.0159483	valid_1's binary_logloss: 0.0166615
[529]	training's binary_logloss: 0.0159317	valid_1's binary_logloss: 0.0166522
[530]	training's binary_logloss: 0.015913	valid_1's binary_logloss: 0.0166436
[531]	training's binary_logloss: 0.0158939	valid_1's binary_logloss: 0.0166341
[532]	training's binary_logloss: 0.0158759	valid_1's binary_logloss: 0.0166232
[533]	training's binary_logloss: 0.0158589	valid_1's binary_logloss: 0.0166145
[534]	training's binary_logloss: 0.0158408	valid_1's binary_logloss: 0.0166054
[535]	training's binary_logloss: 0.015824	valid_1's binary_logloss: 0.0165961
[536]	training's binary_logloss: 0.0158081	valid_1's binary_logloss: 0.0165872
[537]	training's binary_logloss: 0.0157931	valid_1's binary_logloss: 0.0165776
[538]	training's binary_logloss: 0.0157751	valid_1's binary_logloss: 0.0165674
[539]	training's binary_logloss: 0.0157584	valid_1's binary_logloss: 0.0165582
[540]	training's binary_logloss: 0.0157438	valid_1's b

[633]	training's binary_logloss: 0.0145286	valid_1's binary_logloss: 0.0159626
[634]	training's binary_logloss: 0.0145204	valid_1's binary_logloss: 0.0159592
[635]	training's binary_logloss: 0.0145105	valid_1's binary_logloss: 0.0159564
[636]	training's binary_logloss: 0.0145018	valid_1's binary_logloss: 0.0159531
[637]	training's binary_logloss: 0.0144932	valid_1's binary_logloss: 0.0159486
[638]	training's binary_logloss: 0.0144843	valid_1's binary_logloss: 0.0159445
[639]	training's binary_logloss: 0.0144763	valid_1's binary_logloss: 0.0159414
[640]	training's binary_logloss: 0.0144659	valid_1's binary_logloss: 0.0159389
[641]	training's binary_logloss: 0.0144577	valid_1's binary_logloss: 0.0159369
[642]	training's binary_logloss: 0.0144477	valid_1's binary_logloss: 0.0159344
[643]	training's binary_logloss: 0.0144408	valid_1's binary_logloss: 0.0159325
[644]	training's binary_logloss: 0.0144297	valid_1's binary_logloss: 0.0159298
[645]	training's binary_logloss: 0.0144186	valid_1's

[738]	training's binary_logloss: 0.0136983	valid_1's binary_logloss: 0.0156755
[739]	training's binary_logloss: 0.0136904	valid_1's binary_logloss: 0.0156743
[740]	training's binary_logloss: 0.013685	valid_1's binary_logloss: 0.0156731
[741]	training's binary_logloss: 0.0136781	valid_1's binary_logloss: 0.015669
[742]	training's binary_logloss: 0.013671	valid_1's binary_logloss: 0.0156661
[743]	training's binary_logloss: 0.0136637	valid_1's binary_logloss: 0.0156627
[744]	training's binary_logloss: 0.0136566	valid_1's binary_logloss: 0.0156622
[745]	training's binary_logloss: 0.0136503	valid_1's binary_logloss: 0.0156601
[746]	training's binary_logloss: 0.013645	valid_1's binary_logloss: 0.015659
[747]	training's binary_logloss: 0.013638	valid_1's binary_logloss: 0.0156575
[748]	training's binary_logloss: 0.0136317	valid_1's binary_logloss: 0.0156556
[749]	training's binary_logloss: 0.0136249	valid_1's binary_logloss: 0.015654
[750]	training's binary_logloss: 0.0136185	valid_1's binary

[842]	training's binary_logloss: 0.0130511	valid_1's binary_logloss: 0.0155614
[843]	training's binary_logloss: 0.0130446	valid_1's binary_logloss: 0.0155605
[844]	training's binary_logloss: 0.013039	valid_1's binary_logloss: 0.0155597
[845]	training's binary_logloss: 0.0130342	valid_1's binary_logloss: 0.0155601
[846]	training's binary_logloss: 0.0130271	valid_1's binary_logloss: 0.0155605
[847]	training's binary_logloss: 0.0130196	valid_1's binary_logloss: 0.0155599
[848]	training's binary_logloss: 0.0130132	valid_1's binary_logloss: 0.0155589
[849]	training's binary_logloss: 0.0130045	valid_1's binary_logloss: 0.0155593
[850]	training's binary_logloss: 0.0129954	valid_1's binary_logloss: 0.0155589
[851]	training's binary_logloss: 0.0129888	valid_1's binary_logloss: 0.0155597
[852]	training's binary_logloss: 0.0129824	valid_1's binary_logloss: 0.0155596
[853]	training's binary_logloss: 0.0129766	valid_1's binary_logloss: 0.0155594
[854]	training's binary_logloss: 0.0129681	valid_1's 

[947]	training's binary_logloss: 0.0122682	valid_1's binary_logloss: 0.015507
[948]	training's binary_logloss: 0.0122616	valid_1's binary_logloss: 0.0155063
[949]	training's binary_logloss: 0.0122575	valid_1's binary_logloss: 0.0155064
[950]	training's binary_logloss: 0.0122535	valid_1's binary_logloss: 0.0155077
[951]	training's binary_logloss: 0.0122446	valid_1's binary_logloss: 0.0155066
[952]	training's binary_logloss: 0.0122358	valid_1's binary_logloss: 0.0155066
[953]	training's binary_logloss: 0.0122291	valid_1's binary_logloss: 0.0155068
[954]	training's binary_logloss: 0.0122192	valid_1's binary_logloss: 0.0155066
[955]	training's binary_logloss: 0.0122099	valid_1's binary_logloss: 0.0155066
[956]	training's binary_logloss: 0.0122026	valid_1's binary_logloss: 0.0155067
[957]	training's binary_logloss: 0.012197	valid_1's binary_logloss: 0.0155064
[958]	training's binary_logloss: 0.0121924	valid_1's binary_logloss: 0.0155056
[959]	training's binary_logloss: 0.0121866	valid_1's b

In [113]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importance(),X_test.columns)), columns=['Value','Feature'])
feature_imp[-100:]


,Value,Feature
1384,17,time_wildebeest
1385,18,c2_269
1386,18,c2_272
1387,18,c3_340
1388,18,c3_984
1389,18,mean_combines_baboon
1390,18,mean_with_combines_imgs_baboon
1391,18,median_img3_lionmale
1392,18,pred_model_480_360_as480_img3_otherbird
1393,18,pred_model_512_384_v2_as512_img2_hippopotamus


In [29]:

clazzs = ['empty', 'wildebeest', 'zebra', 'gazellethomsons', 'buffalo',
'elephant', 'hartebeest', 'impala', 'gazellegrants', 'giraffe',
'warthog', 'guineafowl', 'otherbird']


        
for clazz in clazzs:
    
#     feature_classes=feature_classes_d.get(clazz, list(train_labels_columns))

    columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']



    X_train = train_dataset[columns]
    y_train = train_dataset[clazz]
    X_val = val_dataset[columns]
    y_val = val_dataset[clazz]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': {'binary_logloss'},
                'num_leaves': num_leaves.get(clazz, 32),
                'max_depth': max_depth.get(clazz, 6),
                'learning_rate': learning_rate.get(clazz, 0.009),
                'feature_fraction':  feature_fraction.get(clazz, 0.6),
                'bagging_fraction': bagging_fraction.get(clazz, 0.8),
                'bagging_freq': bagging_freq.get(clazz, 5),
                'scale_pos_weight':scale_pos_weight.get(clazz, 1),
                'lambda_l1':lambda_l1.get(clazz, 0.2),
                'lambda_l2':lambda_l2.get(clazz, 0.2),
                'min_child_samples':min_child_samples.get(clazz, 25),

    }

    gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=3000,
                        valid_sets=[lgb_train, lgb_val],
                        early_stopping_rounds=20)
    gbm.save_model("lgbs_v12/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

    X_test = test_dataset[columns]
    y_test = test_dataset[clazz]
    
    print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
    gc.collect()


[1]	training's binary_logloss: 0.512186	valid_1's binary_logloss: 0.512086
Training until validation scores don't improve for 20 rounds
[2]	training's binary_logloss: 0.504866	valid_1's binary_logloss: 0.504834
[3]	training's binary_logloss: 0.497772	valid_1's binary_logloss: 0.497807
[4]	training's binary_logloss: 0.490873	valid_1's binary_logloss: 0.490971
[5]	training's binary_logloss: 0.484174	valid_1's binary_logloss: 0.484349
[6]	training's binary_logloss: 0.477653	valid_1's binary_logloss: 0.477894
[7]	training's binary_logloss: 0.471303	valid_1's binary_logloss: 0.471612
[8]	training's binary_logloss: 0.465129	valid_1's binary_logloss: 0.46551
[9]	training's binary_logloss: 0.459118	valid_1's binary_logloss: 0.459581
[10]	training's binary_logloss: 0.453241	valid_1's binary_logloss: 0.453769
[11]	training's binary_logloss: 0.447538	valid_1's binary_logloss: 0.448133
[12]	training's binary_logloss: 0.441947	valid_1's binary_logloss: 0.442611
[13]	training's binary_logloss: 0.436

[109]	training's binary_logloss: 0.176332	valid_1's binary_logloss: 0.180183
[110]	training's binary_logloss: 0.174946	valid_1's binary_logloss: 0.178812
[111]	training's binary_logloss: 0.17358	valid_1's binary_logloss: 0.177467
[112]	training's binary_logloss: 0.172224	valid_1's binary_logloss: 0.176127
[113]	training's binary_logloss: 0.170895	valid_1's binary_logloss: 0.174825
[114]	training's binary_logloss: 0.169615	valid_1's binary_logloss: 0.173571
[115]	training's binary_logloss: 0.168302	valid_1's binary_logloss: 0.172277
[116]	training's binary_logloss: 0.167007	valid_1's binary_logloss: 0.171
[117]	training's binary_logloss: 0.165723	valid_1's binary_logloss: 0.169728
[118]	training's binary_logloss: 0.16445	valid_1's binary_logloss: 0.168474
[119]	training's binary_logloss: 0.163191	valid_1's binary_logloss: 0.167227
[120]	training's binary_logloss: 0.161944	valid_1's binary_logloss: 0.165997
[121]	training's binary_logloss: 0.160714	valid_1's binary_logloss: 0.164783
[122

[216]	training's binary_logloss: 0.0849069	valid_1's binary_logloss: 0.0904527
[217]	training's binary_logloss: 0.0844104	valid_1's binary_logloss: 0.0899693
[218]	training's binary_logloss: 0.083919	valid_1's binary_logloss: 0.0894928
[219]	training's binary_logloss: 0.0834309	valid_1's binary_logloss: 0.0890147
[220]	training's binary_logloss: 0.0829456	valid_1's binary_logloss: 0.0885392
[221]	training's binary_logloss: 0.0824721	valid_1's binary_logloss: 0.0880826
[222]	training's binary_logloss: 0.0819941	valid_1's binary_logloss: 0.08762
[223]	training's binary_logloss: 0.0815201	valid_1's binary_logloss: 0.0871562
[224]	training's binary_logloss: 0.0810563	valid_1's binary_logloss: 0.0867018
[225]	training's binary_logloss: 0.0805886	valid_1's binary_logloss: 0.0862446
[226]	training's binary_logloss: 0.0801278	valid_1's binary_logloss: 0.0857918
[227]	training's binary_logloss: 0.0796686	valid_1's binary_logloss: 0.0853408
[228]	training's binary_logloss: 0.079219	valid_1's bin

[321]	training's binary_logloss: 0.0503394	valid_1's binary_logloss: 0.057033
[322]	training's binary_logloss: 0.0501459	valid_1's binary_logloss: 0.0568529
[323]	training's binary_logloss: 0.0499419	valid_1's binary_logloss: 0.0566625
[324]	training's binary_logloss: 0.0497362	valid_1's binary_logloss: 0.0564701
[325]	training's binary_logloss: 0.0495349	valid_1's binary_logloss: 0.0562739
[326]	training's binary_logloss: 0.0493368	valid_1's binary_logloss: 0.0560848
[327]	training's binary_logloss: 0.0491379	valid_1's binary_logloss: 0.0558979
[328]	training's binary_logloss: 0.0489447	valid_1's binary_logloss: 0.0557109
[329]	training's binary_logloss: 0.0487518	valid_1's binary_logloss: 0.0555298
[330]	training's binary_logloss: 0.048559	valid_1's binary_logloss: 0.0553454
[331]	training's binary_logloss: 0.0483662	valid_1's binary_logloss: 0.0551635
[332]	training's binary_logloss: 0.0481802	valid_1's binary_logloss: 0.0549902
[333]	training's binary_logloss: 0.0479903	valid_1's b

[425]	training's binary_logloss: 0.0357268	valid_1's binary_logloss: 0.0434766
[426]	training's binary_logloss: 0.0356343	valid_1's binary_logloss: 0.0433928
[427]	training's binary_logloss: 0.0355415	valid_1's binary_logloss: 0.043309
[428]	training's binary_logloss: 0.0354522	valid_1's binary_logloss: 0.0432313
[429]	training's binary_logloss: 0.0353622	valid_1's binary_logloss: 0.0431511
[430]	training's binary_logloss: 0.0352725	valid_1's binary_logloss: 0.0430716
[431]	training's binary_logloss: 0.0351846	valid_1's binary_logloss: 0.0429956
[432]	training's binary_logloss: 0.0350951	valid_1's binary_logloss: 0.0429137
[433]	training's binary_logloss: 0.0350083	valid_1's binary_logloss: 0.0428391
[434]	training's binary_logloss: 0.0349225	valid_1's binary_logloss: 0.0427585
[435]	training's binary_logloss: 0.0348363	valid_1's binary_logloss: 0.0426841
[436]	training's binary_logloss: 0.0347508	valid_1's binary_logloss: 0.0426069
[437]	training's binary_logloss: 0.0346678	valid_1's 

...

[516]	training's binary_logloss: 0.000226003	valid_1's binary_logloss: 0.000854141
[517]	training's binary_logloss: 0.000225076	valid_1's binary_logloss: 0.000854016
[518]	training's binary_logloss: 0.00022422	valid_1's binary_logloss: 0.000853906
[519]	training's binary_logloss: 0.00022332	valid_1's binary_logloss: 0.000853692
[520]	training's binary_logloss: 0.000222397	valid_1's binary_logloss: 0.000853887
[521]	training's binary_logloss: 0.000221621	valid_1's binary_logloss: 0.000853841
[522]	training's binary_logloss: 0.000220725	valid_1's binary_logloss: 0.00085386
[523]	training's binary_logloss: 0.000219726	valid_1's binary_logloss: 0.000853787
[524]	training's binary_logloss: 0.000218894	valid_1's binary_logloss: 0.000853717
[525]	training's binary_logloss: 0.000218018	valid_1's binary_logloss: 0.000853697
[526]	training's binary_logloss: 0.000217121	valid_1's binary_logloss: 0.00085368
[527]	training's binary_logloss: 0.000216241	valid_1's binary_logloss: 0.000853286
[528]	tr

[14]	training's binary_logloss: 0.0136791	valid_1's binary_logloss: 0.0163995
[15]	training's binary_logloss: 0.0134768	valid_1's binary_logloss: 0.0162797
[16]	training's binary_logloss: 0.013285	valid_1's binary_logloss: 0.0161273
[17]	training's binary_logloss: 0.0131018	valid_1's binary_logloss: 0.0159947
[18]	training's binary_logloss: 0.0129311	valid_1's binary_logloss: 0.0158697
[19]	training's binary_logloss: 0.012772	valid_1's binary_logloss: 0.0157469
[20]	training's binary_logloss: 0.0126172	valid_1's binary_logloss: 0.0156285
[21]	training's binary_logloss: 0.0124578	valid_1's binary_logloss: 0.0155041
[22]	training's binary_logloss: 0.0123097	valid_1's binary_logloss: 0.01539
[23]	training's binary_logloss: 0.0121661	valid_1's binary_logloss: 0.015277
[24]	training's binary_logloss: 0.0120275	valid_1's binary_logloss: 0.0151658
[25]	training's binary_logloss: 0.0118929	valid_1's binary_logloss: 0.0150602
[26]	training's binary_logloss: 0.0117689	valid_1's binary_logloss: 0

[119]	training's binary_logloss: 0.00683534	valid_1's binary_logloss: 0.0115464
[120]	training's binary_logloss: 0.006807	valid_1's binary_logloss: 0.0115345
[121]	training's binary_logloss: 0.00678023	valid_1's binary_logloss: 0.0115187
[122]	training's binary_logloss: 0.00675469	valid_1's binary_logloss: 0.0115035
[123]	training's binary_logloss: 0.00673091	valid_1's binary_logloss: 0.0114877
[124]	training's binary_logloss: 0.00670544	valid_1's binary_logloss: 0.0114723
[125]	training's binary_logloss: 0.00668232	valid_1's binary_logloss: 0.0114577
[126]	training's binary_logloss: 0.00665761	valid_1's binary_logloss: 0.0114416
[127]	training's binary_logloss: 0.00663392	valid_1's binary_logloss: 0.011428
[128]	training's binary_logloss: 0.00661027	valid_1's binary_logloss: 0.0114123
[129]	training's binary_logloss: 0.00658846	valid_1's binary_logloss: 0.0113967
[130]	training's binary_logloss: 0.00656467	valid_1's binary_logloss: 0.0113834
[131]	training's binary_logloss: 0.00654168

[223]	training's binary_logloss: 0.00501029	valid_1's binary_logloss: 0.0107417
[224]	training's binary_logloss: 0.00499655	valid_1's binary_logloss: 0.0107371
[225]	training's binary_logloss: 0.00498629	valid_1's binary_logloss: 0.0107347
[226]	training's binary_logloss: 0.00497369	valid_1's binary_logloss: 0.0107291
[227]	training's binary_logloss: 0.00496134	valid_1's binary_logloss: 0.0107239
[228]	training's binary_logloss: 0.00494987	valid_1's binary_logloss: 0.0107186
[229]	training's binary_logloss: 0.00493957	valid_1's binary_logloss: 0.0107143
[230]	training's binary_logloss: 0.00492822	valid_1's binary_logloss: 0.0107096
[231]	training's binary_logloss: 0.00491779	valid_1's binary_logloss: 0.0107088
[232]	training's binary_logloss: 0.00490721	valid_1's binary_logloss: 0.0107051
[233]	training's binary_logloss: 0.00489664	valid_1's binary_logloss: 0.0107031
[234]	training's binary_logloss: 0.00488587	valid_1's binary_logloss: 0.0106999
[235]	training's binary_logloss: 0.00487

[326]	training's binary_logloss: 0.00410808	valid_1's binary_logloss: 0.0105357
[327]	training's binary_logloss: 0.0041017	valid_1's binary_logloss: 0.0105338
[328]	training's binary_logloss: 0.00409599	valid_1's binary_logloss: 0.010532
[329]	training's binary_logloss: 0.0040885	valid_1's binary_logloss: 0.0105306
[330]	training's binary_logloss: 0.0040818	valid_1's binary_logloss: 0.0105304
[331]	training's binary_logloss: 0.00407463	valid_1's binary_logloss: 0.0105303
[332]	training's binary_logloss: 0.0040691	valid_1's binary_logloss: 0.0105305
[333]	training's binary_logloss: 0.00406208	valid_1's binary_logloss: 0.0105294
[334]	training's binary_logloss: 0.00405582	valid_1's binary_logloss: 0.010529
[335]	training's binary_logloss: 0.00404918	valid_1's binary_logloss: 0.0105287
[336]	training's binary_logloss: 0.00404413	valid_1's binary_logloss: 0.0105287
[337]	training's binary_logloss: 0.00403843	valid_1's binary_logloss: 0.0105294
[338]	training's binary_logloss: 0.00403139	va

In [27]:
# Early stopping, best iteration is:
# [909]	training's binary_logloss: 0.0253938	valid_1's binary_logloss: 0.0325604
# empty 0.032561336883759374
# Early stopping, best iteration is:
# [845]	training's binary_logloss: 0.00949369	valid_1's binary_logloss: 0.0205298
# wildebeest 0.02134952140600164
# Early stopping, best iteration is:
# [1190]	training's binary_logloss: 0.00587704	valid_1's binary_logloss: 0.0100063
# zebra 0.010410163679976766
# Early stopping, best iteration is:
# [837]	training's binary_logloss: 0.00760207	valid_1's binary_logloss: 0.00604578
# gazellethomsons 0.006370004320359023
# Early stopping, best iteration is:
# [594]	training's binary_logloss: 0.00131993	valid_1's binary_logloss: 0.00478458
# buffalo 0.0045945335607014
# Early stopping, best iteration is:
# [661]	training's binary_logloss: 0.000644675	valid_1's binary_logloss: 0.00358995
# elephant 0.0037806150024401186
# Early stopping, best iteration is:
# [539]	training's binary_logloss: 0.000635327	valid_1's binary_logloss: 0.00310088
# hartebeest 0.003238209560658141
# Early stopping, best iteration is:
# [559]	training's binary_logloss: 0.00120099	valid_1's binary_logloss: 0.0039591
# impala 0.004122439429679086
# Early stopping, best iteration is:
# [515]	training's binary_logloss: 0.00315171	valid_1's binary_logloss: 0.00524783
# gazellegrants 0.004874318450402127
steps={
    'empty' : int(910),
    'wildebeest' : int(850),
    'zebra' : int(1200),
    'gazellethomsons' : int(900),
    'buffalo' : int(600),
    'elephant' : int(660),
    'hartebeest' : int(540),
    'impala' : int(600),
    'gazellegrants' : int(600),
    'giraffe' : int(705*1.1),
    'warthog' : int(752*1.1),
    'guineafowl' : int(416*1.1),
    'otherbird' : int(348*1.1),


}

138